In [1]:
from dotenv import load_dotenv
from snowflake.snowpark.session import Session
import os
from snowflake.core import Root
from typing import List

C:\Users\Administrator\AppData\Local\Temp\ipykernel_17796\824325259.py:2: DeprecationWarning: Python Runtime 3.8 reached its End-Of-Life (EOL) on October 14, 2024, there will be no further bug fixes or security updates for this runtime. We recommend that you upgrade your existing Python 3.8 objects to Python 3.9, 3.10 or 3.11 before March 31, 2025. Please note that end of support does not impact execution, and you will still be able to update and invoke existing objects. However, they will be running on an unsupported runtime which will no longer be maintained or patched by the Snowflake team. For more details, please refer to https://docs.snowflake.com/en/developer-guide/python-runtime-support-policy.
  from snowflake.snowpark.session import Session


In [2]:
load_dotenv(override=True)

connection_params = {
  "account":  os.environ["SNOWFLAKE_ACCOUNT"],
  "user": os.environ["SNOWFLAKE_USER"],
  "password": os.environ["SNOWFLAKE_USER_PASSWORD"],
  "role": os.environ["SNOWFLAKE_ROLE"],
  "database": os.environ["SNOWFLAKE_DATABASE"],
  "schema": os.environ["SNOWFLAKE_SCHEMA"],
  "warehouse": os.environ["SNOWFLAKE_WAREHOUSE"],
}

snowpark_session = Session.builder.configs(connection_params).create()

In [3]:
class CortexSearchRetriever:

    def __init__(self, snowpark_session: Session, limit_to_retrieve: int = 4):
        self._snowpark_session = snowpark_session
        self._limit_to_retrieve = limit_to_retrieve

    def retrieve(self, query: str) -> List[str]:
        root = Root(self._snowpark_session)
        cortex_search_service = (
            root.databases[os.environ["SNOWFLAKE_DATABASE"]]
            .schemas[os.environ["SNOWFLAKE_SCHEMA"]]
            .cortex_search_services[os.environ["SNOWFLAKE_CORTEX_SEARCH_SERVICE"]]
        )
        resp = cortex_search_service.search(
            query=query,
            columns=["INTRODUCTION"],
            limit=self._limit_to_retrieve,
        )

        if resp.results:
            return [curr["INTRODUCTION"] for curr in resp.results]
        else:
            return []

In [4]:
retriever = CortexSearchRetriever(snowpark_session=snowpark_session, limit_to_retrieve=4)

In [5]:
retrieved_context = retriever.retrieve(query="the largest city in")
print(retrieved_context)

['Wichita is the largest city in Kansas — since Kansas City is divided between the states of Kansas and Missouri. It is not the biggest or flashiest city in the US, but it is a medium-sized city with a small-town feel. It offers much to tourists from both near and far with its variety of attractions.', 'Guayaquil is the largest city in Ecuador. This article also covers Durán, which is a UNESCO Creative City on the other side of the river.', 'Timika is the largest city in Central Papua province in Papua, with 142,000 residents in 2020. It has become a gateway to one of the largest gold mines on the planet, the Grasberg mine.', 'Domodossola is the largest city in the Ossola district, and the second-largest city in the province of Verbano-Cusio-Ossola, in Piedmont, in Northwest Italy.']


In [6]:
print(len(retrieved_context))

4


In [7]:
snowpark_session.close()